### Programming assignment: 

#### Produce an updated 2D posterior distribution after each movement and subsequent measurement - Localisation.

The function localize takes the following arguments:

Colours:
       2D list, each entry either 'R' (for red cell) or 'G' (for green cell)

Measurements:
       list of measurements taken by the robot, each entry either 'R' or 'G'

Motions:
       list of actions taken by the robot, each entry of the form [dy,dx],
       where dx refers to the change in the x-direction (positive meaning
       movement to the right) and dy refers to the change in the y-direction
       (positive meaning movement downward)
       NOTE: the *first* coordinate is change in y; the *second* coordinate is
             change in x

sensor_right:
       float between 0 and 1, giving the probability that any given
       measurement is correct; the probability that the measurement is
       incorrect is 1-sensor_right

p_move:
       float between 0 and 1, giving the probability that any given movement
       command takes place; the probability that the movement command fails
       (and the robot remains still) is 1-p_move; the robot will NOT overshoot
       its destination in this exercise

The function should RETURN (not just show or print) a 2D list (of the same
dimensions as colors) that gives the probabilities that the robot occupies
each cell in the world.

Compute the probabilities by assuming the robot initially has a uniform
probability of being in any cell.

Also assume that at each step, the robot:
1) first makes a movement,
2) then takes a measurement.

Motion:
 [0,0] - stay
 [0,1] - right
 [0,-1] - left
 [1,0] - down
 [-1,0] - up

In [39]:
import numpy as np

def localize(colors,measurements,motions,sensor_right,p_move):
    # initializes p to a uniform distribution over a grid of the same dimensions as colors
    pinit = 1.0 / float(len(colors)) / float(len(colors[0]))
    p = [[pinit for row in range(len(colors[0]))] for col in range(len(colors))]
    
    for i,step in enumerate(motions):
        p = move(step,p_move,p)
        p = sense(measurements[i],sensor_right,p,colors)
    return p

def move(motion,p_move,p):
    # `motions` of type [down(up),right(left)]
    try:
        assert isintance(p,np.ndarray)
    except:
        p = np.array(p)
    cache = np.zeros(p.shape) # the result if movement happens
    for i in range(p.shape[0]): # rows
        for j in range(p.shape[1]):# columns
            cache[i,j] = p[(i-motion[0])%p.shape[0],(j-motion[1])%p.shape[1]]
    # Take into account probability of no movement:
    p = p_move*np.array(cache) + (1-p_move)*p 
    prob_sum = sum(map(sum,p)) # `map(sum,colors)` sums each row and then the row sums summed over columns
    p = p/prob_sum
    return p

def sense(measurement,sensor_right,p,colors):
    # Uses Bayesian inference formula
    try:
        assert isintance(p,np.ndarray)
    except:
        p = np.array(p)
    non_norm_posterior = np.zeros(p.shape)
    for i in range(p.shape[0]):
        for j in range(p.shape[1]):
            true_hit = measurement == colors[i][j]
            non_norm_posterior[i,j] = (sensor_right*true_hit+(1-sensor_right)*(1-true_hit)) * p[i,j]
    prob_sum = sum(map(sum,non_norm_posterior))
    p = non_norm_posterior/prob_sum
    return p

def show(p):
    rows = ['[' + ','.join(map(lambda x: '{0:.5f}'.format(x),r)) + ']' for r in p]
    print('[' + ',\n '.join(rows) + ']')

In [40]:
## Test:
colors = [['R','G','G','R','R'],
          ['R','R','G','R','R'],
          ['R','R','G','G','R'],
          ['R','R','R','R','R']]
measurements = ['G','G','G','G','G']
motions = [[0,0],[0,1],[1,0],[1,0],[0,1]]
p = localize(colors,measurements,motions,sensor_right = 0.7, p_move = 0.8)
show(p) # displays your answer

[[0.01106,0.02464,0.06800,0.04472,0.02465],
 [0.00715,0.01017,0.08697,0.07988,0.00935],
 [0.00740,0.00894,0.11273,0.35351,0.04066],
 [0.00911,0.00715,0.01435,0.04313,0.03643]]
